In [ ]:

from typing import (    
    Any    
)


from conf.constants import QDRANT_KEY, QDRANT_URL
from openai import OpenAI
from qdrant_client import QdrantClient

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np

from langchain_community.vectorstores.utils import maximal_marginal_relevance
from langchain_core.documents import Document
from langchain_core.embeddings import Embeddings
from langchain_core.vectorstores import VectorStore


# create an embedding using openai
def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   resp = OpenAI().embeddings.create(input = [text], model=model)
   return resp.data[0].embedding

def document_from_scored_point(        
        scored_point: Any,
        content_payload_key: str,
        metadata_payload_key: str,
    ) -> Document:
        return Document(
            page_content=scored_point.payload.get(content_payload_key),
            metadata=scored_point.payload.get(metadata_payload_key) or {},
        )
    
# query the vector store
def query_qdrant(query, collection_name, top_k=20):
    
    embedding = get_embedding(text=query)

    qdrant_client = QdrantClient(
        QDRANT_URL,
        api_key=QDRANT_KEY,
    )

    results = qdrant_client.search(
        collection_name=collection_name,
        query_vector=(embedding),
        with_payload=True,
        with_vectors=True,
        limit=top_k,
    )

    ## The MMR impl used with retriever(search_type='mmr')    
    embeddings = [result.vector for result in results]
    
    mmr_selected = maximal_marginal_relevance(
            np.array(embedding), embeddings, k=5, lambda_mult=0.75
        )
    
    return [
        (
            document_from_scored_point(
                scored_point=results[i], 
                content_payload_key="page_content", 
                metadata_payload_key="metadata"
            ),
            results[i].score,
        )
        for i in mmr_selected
    ]


In [79]:
COLLECTION = "rhaetor.github.io_components_2"
QUERY = "accept messages while it is stopping"
NUM_RESULTS = 5
query_results = query_qdrant(        
    query=QUERY, 
    collection_name=COLLECTION,
    top_k=NUM_RESULTS
    )

df = pd.DataFrame(columns=['score', 'page_ref', 'entities', 'content', 'content_size'])

for i, article in enumerate(query_results):    
    doc = article[0]
    score = article[1]
    
    data = {        
        "score": round(score, 3),
        "page_ref": doc.metadata["page_number"],
        "entities": doc.metadata["entities"],
        "content": doc.page_content,
        "content_size": len(doc.page_content)              
    }
    df_new_rows = pd.DataFrame(data, index=[i])
    df = pd.concat([df, df_new_rows])   
    
print("done")

done


# General structure

In [80]:
display(df.iloc[:NUM_RESULTS])

,score,page_ref,entities,content,content_size
0,0.755,_eips_enterprise-integration-patterns.html.txt_3,The top 20 entities mentioned in the given con...,"""Resumable Consumer\nHow can a message\nreceiv...",1401
1,0.748,_eips_split-eip.html.txt_1,The top 20 entities mentioned in the given con...,"""false\nBoolean\n\n\nparallelProcessing\n\nIf ...",2272
2,0.751,_next_activemq-component.html.txt_4,The top 20 entities mentioned in the given con...,"""selector (consumer)\n\nSets the JMS selector ...",2406
3,0.749,_eips_multicast-eip.html.txt_4,The top 20 entities mentioned in the given con...,"""But sometimes you just want Camel to stop and...",2115
4,0.752,_next_amqp-component.html.txt_19,The top 20 entities mentioned in the given con...,"""String\n\n\n\nsubscriptionShared (consumer)\n...",2462


# Entities

In [77]:
entities = df['entities'].tolist()
for i, item in enumerate(entities):        
    print("Item", str(i),": \n", item, "\n\n")

Item 0 : 
 The top 20 entities mentioned in the given context are:

1. subscriptionShared
2. subscriptionName
3. subscriptionDurable
4. pub-sub domain
5. JMS 2.0
6. acceptMessagesWhileStopping
7. JMS route
8. JMS broker
9. dead letter queue
10. reply managers
11. request-reply messaging
12. CamelContext
13. consumerType
14. Simple
15. Default
16. Custom
17. DefaultMessageListenerContainer
18. SimpleMessageListenerContainer
19. DefaultTaskExecutorType
20. MessageListenerContainerFactory

Summary:
The context discusses various advanced consumer options in the context of JMS (Java Message Service) and Apache Camel. It includes settings related to subscription, message acceptance while stopping, reply managers, consumer types, and task execution. These settings are crucial for configuring JMS routes and ensuring smooth message processing within the JMS environment. 


Item 1 : 
 The top 20 entities mentioned in the given context are:
1. resultMinimumWaitTime
2. assertIsSatisfied()
3. latch